# HII region expansion in $1 / r^2$ density profile

## TOC
* [Packages & Settings](#packages)
* [Snapshots](#snapshots)
* [Loading Snapshots](#loading)
* [HI maps](#HI-maps)
* [I-front position](#pos-vel)

## Packages & Settings  <a class="anchor" id="packages"></a>

In [ ]:
from pyrhyme import PyRhyme

from pathlib import Path

# !pip install numpy
import numpy as np

# !pip install astropy
from astropy import units as U
from astropy import constants as C
from astropy.cosmology import WMAP7

# !pip install scipy
from scipy.io import FortranFile
from scipy.interpolate import interp1d

# !pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"

## Snapshots  <a class="anchor" id="snapshots"></a>

Run
```bash
wget -r –level=0 -E –ignore-length -x -k -p -erobots=off -np -N https://astronomy.sussex.ac.uk/\~iti20/RT_comparison_project/RT_workshop_data/T6_results/
```
to download Iliev-5 test snapshots!

In [ ]:
ILIEV_5_DIR = 'astronomy.sussex.ac.uk/~iti20/RT_comparison_project/RT_workshop_data/T6_results'

IMAGE_DIR = './images'
Path(IMAGE_DIR).mkdir(parents=True, exist_ok=True)
TIMES = [1, 3, 10, 25, 75]

BINARIES = {
    'CAPREOLE+C2-RAY': {
        'orig': {t: {'path': f"{ILIEV_5_DIR}/c2ray{i+1}.bin"} for i, t in enumerate(TIMES)},
        'add': {t: {'path': f"{ILIEV_5_DIR}/c2ray_add{i+1}.bin"} for i, t in enumerate(TIMES)},
    },
#     'Flash': {
#         'orig': {t: {'path': f"{ILIEV_5_DIR}/flash{i+1}.bin"} for i, t in enumerate(TIMES)},
#         'add': {t: {'path': f"{ILIEV_5_DIR}/flash_add{i+1}.bin"} for i, t in enumerate(TIMES)},
#     },
    'HART': {
        'orig': {t: {'path': f"{ILIEV_5_DIR}/hart{i+1}.bin"} for i, t in enumerate(TIMES)},
        'add': {t: {'path': f"{ILIEV_5_DIR}/hart_add{i+1}.bin"} for i, t in enumerate(TIMES)},
    },
#     'LICORICE': {
#         'orig': {t: {'path': f"{ILIEV_5_DIR}/licorice{i+1}.bin"} for i, t in enumerate(TIMES)},
#         'add': {t: {'path': f"{ILIEV_5_DIR}/licorice_add{i+1}.bin"} for i, t in enumerate(TIMES)},
#     },
    'RSPH': {
        'orig': {t: {'path': f"{ILIEV_5_DIR}/rsph{i+1}.bin"} for i, t in enumerate(TIMES)},
        'add': {t: {'path': f"{ILIEV_5_DIR}/rsph_add{i+1}.bin"} for i, t in enumerate(TIMES)},
    },
    'RH1D': {
        'orig': {t: {'path': f"{ILIEV_5_DIR}/rh1d{i+1}.bin"} for i, t in enumerate(TIMES)},
        'add': {t: {'path': f"{ILIEV_5_DIR}/rh1d_add{i+1}.bin"} for i, t in enumerate(TIMES)},
    },
#     'ZEUS-MP': {
#         'orig': {t: {'path': f"{ILIEV_5_DIR}/zeus{i+1}.bin"} for i, t in enumerate(TIMES)},
#         'add': {t: {'path': f"{ILIEV_5_DIR}/zeus_add{i+1}.bin"} for i, t in enumerate(TIMES)},
#     },
}

Rhyme_CaseA = {
    0: {'path': f'./Iliev-6-CaseA-PowerLaw/Iliev-6-CaseA-000000.chombo.h5', },
}

Rhyme_CaseB = {
    0: {'path': f'./Iliev-6-CaseB-PowerLaw/Iliev-6-CaseB-000000.chombo.h5', },
}

# Check if files exist
def check_if_exist(path):
    if not Path(snap['path']).is_file():
        print(f"Not found: {snap['path']}")
        
for sim_name, sim in BINARIES.items():
    for snap in sim['orig'].values():
        check_if_exist(snap['path'])
    for snap in sim['add'].values():
        check_if_exist(snap['path'])

for snap in Rhyme_CaseA.values():
    check_if_exist(snap['path'])
for snap in Rhyme_CaseB.values():
    check_if_exist(snap['path'])

## Loading snapshots  <a class="anchor" id="loading"></a>

In [ ]:
import os
def time_to_snap_id(t, ds):
    for i in range(0, ds.dataset.num_of_snapshots):
        ds.dataset.jump_to(i)
        snap_time = ds.dataset.time
        if snap_time > t:
            if i == 0:
                return i
            
            ds.dataset.jump_to(i-1)
            prev_time = ds.dataset.time
            
            return i if abs(t - snap_time) < abs(t - prev_time) else i - 1
        
    return ds.dataset.num_of_snapshots - 1


def load_rhyme(snap_id, rhyme, Gamma=5./3.):
    rhyme.dataset.jump_to(snap_id)
    res = {}

    v = rhyme.load_variables(silent=True)
    domain = rhyme.dataset.problem_domain

    rho = v['rho'][0].reshape(domain) * (1 * v['rho'][1]).to(U.cm**-3).value
    vx = v['rho_u'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    vy = v['rho_v'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    vz = v['rho_w'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    T_orig = v['temp'][0].reshape(domain)
    fHI = v['ntr_frac_0'][0].reshape(domain)
    p, _, T = rhyme.calc_temperature(v, X=1.0, Y=0.0, Gamma=5./3.)
    T = T[0].reshape(domain)
    p = p.reshape(domain)
    p *= C.m_p.to(U.g).value * (1 * rhyme.dataset.active['h5']['attrs']['pressure_unit']).to(U.cm**-3 * U.cm**2 / U.s**2).value

    M = np.sqrt(vx**2 + vy**2 + vz**2) / np.sqrt(Gamma * p / (C.m_p.to(U.g).value * rho))

    res['n'] = rho
    res['nHI'] = fHI
    res['nHII'] = 1.0 - fHI
    res['T'] = T_orig
    res['p'] = p
    res['M'] = M
    
    return res


def reading_binary_files():
    result = {}
    
    for simname, sim in BINARIES.items():
        for snap_time, snap in sim['orig'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}
                
            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['nHI'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['p'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['T'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
            
        for snap_time, snap in sim['add'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}

            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['n'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['M'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['nHII'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
        
    rA = PyRhyme(Rhyme_CaseA[0]['path'])
    rB = PyRhyme(Rhyme_CaseB[0]['path'])
    
    for t in TIMES:
        result[t]['Rhyme_CaseA'] = load_rhyme(time_to_snap_id(t, rA), rA)
        result[t]['Rhyme_CaseB'] = load_rhyme(time_to_snap_id(t, rB), rB)

    rA.dataset.close_current()
    rA.dataset.clean_all()
    
    rB.dataset.close_current()
    rB.dataset.clean_all()
    
    return result
            
    
if __name__ == '__main__' and '__file__' not in globals():
    DATA = reading_binary_files()

In [ ]:
def check_cell(attr='n', x=1, y=1, z=1):
    for t, snap in DATA.items():
        print('')
        for simname, sim in snap.items():
            print(f"{t:3.1f}: {simname:18s}: {attr} = {sim[attr][x, y, z]:3.2e}")

if __name__ == '__main__' and '__file__' not in globals():
    check_cell(attr='T', x=64)

## HI maps <a class="anchor" id="HI-maps"></a>

In [ ]:
def HI_maps(t=100):
    if t not in DATA.keys():
        print(f"Don't have a snapshot at t = {t} Myr")
        return
    for sn, sim in DATA[t]:
        pass
        

## I-front position <a class="anchor" id="pos-vel"></a>

In [ ]:
def i_front_get_position(xs, fHI, f0=.5):
    ydir = np.sign(np.diff(fHI))
    turning_points = 1 + np.where(np.diff(ydir) != 0)[0]
    
    signs = np.split(ydir, turning_points)
    xs_grp = np.split(xs, turning_points)
    fHI_grp = np.split(fHI, turning_points)
    fs = [
        (interp1d(y, x, bounds_error=False, fill_value=-1), sgn[0])
        for x, y, sgn in zip(xs_grp, fHI_grp, signs) if len(x) > 2
    ]
    
    return [(float(p[0]), p[1]) for p in [(f[0](f0), f[1]) for f in fs] if p[0] != -1]


def i_front_pos():
    xs = np.sqrt(3) * np.linspace(0, 0.8, 128)
    
    for t, snap in DATA.items():
        for sn, sim in DATA[t].items():
            if 'I-front' not in sim:
                sim['I-front'] = {}

            nHI_prof = np.einsum('iii->i', sim['nHI'])
            nHII_prof = np.einsum('iii->i', sim['nHII'])
            fHI_prof = nHII_prof / (nHI_prof + nHII_prof)

            pos = i_front_get_position(xs, fHI_prof, f0=0.9)
            
            if len(pos) == 1:
                sim['I-front']['pos_kpc'] = pos[0][0]
            else:
                sim['I-front']['pos_kpc'] = None
                print(f"{sn}-{t}: {pos}")

if __name__ == '__main__' and '__file__' not in globals():
    i_front_pos()

In [ ]:
def i_front_pos_plot():
    colors = px.colors.qualitative.Plotly # Pastel
    
    fig = go.Figure()
    
    times = list(DATA.keys())
    for i, sn in enumerate(DATA[10].keys()):
        ys = [DATA[t][sn]['I-front']['pos_kpc'] for t in times]
        
        fig.add_trace(go.Scatter(
            x=times, y=ys, name=sn,
            mode='lines+markers',
            line=dict(color=colors[i]),
        ))

            
    width, height = 500 + 150, 500
            
    fig.update_layout(
        margin=dict(b=10, t=10, l=10, r=10),
        width=width, height=height,
        xaxis=dict(
            type='log', mirror=True, title='<i>t</i> [Myr]',
            titlefont=dict(size=18),
            tickmode='array',
            tickvals=TIMES,
            ticktext=[str(tt) for tt in TIMES],
            tickfont=dict(size=14),
        ),
        yaxis=dict(
            type='linear', mirror=True, title='<i>x</i><sub>I-front</sub> [kpc]',
            titlefont=dict(size=18),
            tickmode='array',
            tickvals=np.linspace(0.2, 1.2, 6),
            ticktext=[f"{yy:2.1f}" for yy in np.linspace(0.2, 1.2, 6)],
            tickfont=dict(size=14),
        ),
    )
            
    fig.show()

if __name__ == '__main__' and '__file__' not in globals():
    i_front_pos_plot()